##  running MPI4py on EC2 using spotpricing
#### For general tutorial of running running MPI4py this tutorial is more linked towards spotpricing


In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2



  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-YtAV2O-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


In [2]:
## STARTING UPP LINKER

from ConfigParser import ConfigParser
config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)
from linkingEC2 import LinkingHandler
linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC')

### Here one need to have created some spotprices!

In [3]:
linker.connect_spot_instances()

checking if ssh into node003 works:
*
checking if ssh into node004 works:
*
collecting node003 number of processes:done
collecting node004 number of processes:done
copying ~/.ssh/mykeyABC.rsa to external ~/.ssh/id_rsa
copying files to node003 done
copying files to node004 done
disable StrictHostKeyChecking  in node003:appending to ~/nodefile in node003 done
appending to /etc/hosts in node003 done


In [4]:
import time
PACKAGES_APT = ['python-dev', 'openmpi-bin', 'libopenmpi-dev', 'python-numpy']
PACKAGES_PIP = ['mpi4py']
time_in = time.time()
linker.apt_install(PACKAGES_APT)
linker.pip_install(PACKAGES_PIP)
time_out = time.time()
import os
os.system('say "your packages is downloaded"')
print('it took {time:.2f} sec'.format(time = time_out - time_in))

installing the packages : python-dev openmpi-bin libopenmpi-dev python-numpy

update apt-get for node003
update apt-get for node004
node003 is done
node004 is done
upgrade apt-get for node003
upgrade apt-get for node004
node003 is done
node004 is done
installing the packages : python-pip

installing package for node003
installing package for node004
node003 is done
node004 is done
installing the packages : mpi4py

installing package for node004
installing package for node003
node004 is done
node003 is done
it took 102.44 sec
installing package for node003
installing package for node004
node003 is done
node004 is done


In [5]:
%%writefile calc_pi_MPI.py
from mpi4py import MPI
import numpy as np
import random
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
mpisize = comm.Get_size()
nsamples = int(12e6/mpisize)
inside = 0
random.seed(rank)
for i in range(nsamples):
    x = random.random()
    y = random.random()
    if (x*x)+(y*y)<1:
        inside += 1
mypi = (4.0 * inside)/nsamples
pi = comm.reduce(mypi, op=MPI.SUM, root=0)
if rank==0:
    print('estimate of pi = ',end = '')
    print (1.0 / mpisize)*pi

Overwriting calc_pi_MPI.py


In [6]:
linker.copy_files_to_nodes( file_name = 'calc_pi_MPI.py', destination='~/' ) # deafult desnination is ~/

copying calc_pi_MPI.py to external ~/
copying files to node003 done
copying files to node004 done


In [11]:
import numpy as np
tot_process = np.sum([node['n_process'] for node in linker.nodes])
command = 'mpirun -hostfile nodefile -n %d python calc_pi_MPI.py'%(tot_process)
linker.send_command_ssh(nodes = 0, command = command)

node003, sshing in  mpirun -hostfile nodefile -n 4 python calc_pi_MPI.py 
node003 done


0

In [10]:
print("ssh  -i {keyloc} -o 'StrictHostKeyChecking no'  ubuntu@{hostname}".format(
		keyloc = linker.my_key_location,
		hostname =  linker.nodes[0]['public_dns']))

ssh  -i ~/.ssh/mykeyABC.rsa -o 'StrictHostKeyChecking no'  ubuntu@ec2-52-23-158-28.compute-1.amazonaws.com


In [12]:
linker.terminate_cluster()

TypeError: object of type 'SpotInstanceRequest' has no len()

ERROR:boto:400 Bad Request
ERROR:boto:<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>InvalidSpotInstanceRequestID.Malformed</Code><Message>Invalid id :"SpotInstanceRequest:sir-021j83dz" (expecting "sir-...")</Message></Error></Errors><RequestID>f9238672-5a35-47f7-83a3-c3897ece0aed</RequestID></Response>


EC2ResponseError: EC2ResponseError: 400 Bad Request
<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>InvalidSpotInstanceRequestID.Malformed</Code><Message>Invalid id :"SpotInstanceRequest:sir-021j83dz" (expecting "sir-...")</Message></Error></Errors><RequestID>f9238672-5a35-47f7-83a3-c3897ece0aed</RequestID></Response>